In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
tf.keras.mixed_precision.set_global_policy('mixed_float16')

BATCH_SIZE = 32
CHANNEL_NUMBER = 3
WINDOW_SIZE = 750
SLIDING_STEP = int(WINDOW_SIZE * 0.4)
CLASS_NUMBER = 2
if CLASS_NUMBER < 3:
        CLASS_NUMBER = 1

def slicingAndNormalize(arr):
    def normalizer(X):
        ret = np.zeros(X.shape)
        mean = np.average(X, axis=1)
        std = np.std(X, axis=1)
        ret = (X - mean) / std
        return ret
    
    totalLength = arr.shape[-1]
    if totalLength <= WINDOW_SIZE:
        return arr
    ret = normalizer((arr[:, :WINDOW_SIZE].T)[np.newaxis, :])
    
    i = SLIDING_STEP
    while (totalLength - i) > WINDOW_SIZE:
        new = normalizer((arr[:, i: (i + WINDOW_SIZE)].T)[np.newaxis, :])
        ret = np.concatenate([ret, new], axis=0)
        i += SLIDING_STEP
    return ret

def buildModel():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(8, 6, activation='relu', input_shape=(WINDOW_SIZE, CHANNEL_NUMBER)),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv1D(32, 6, strides=4, activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv1D(48, 6, strides=2, activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv1D(96, 6, activation='relu'),
        tf.keras.layers.BatchNormalization(axis=1),
        tf.keras.layers.MaxPool1D(padding='same'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(CLASS_NUMBER, activation='softmax')]
    )
    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=[tf.keras.metrics.Precision(),
                         tf.keras.metrics.Recall()])
    model.summary()
    return model

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2080 Ti, compute capability 7.5


In [2]:
trainFile = './data/coswave.npy'
X = slicingAndNormalize(np.load(trainFile))
y = np.array([1] * X.shape[0])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=777)

In [3]:
X_train.shape

(18, 750, 3)

In [4]:
a = tf.keras.layers.Conv1D(8, 2, activation='relu', input_shape=(750, 3))(X_train)

In [5]:
model = buildModel()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 745, 8)            152       
                                                                 
 batch_normalization (BatchN  (None, 745, 8)           2980      
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 373, 8)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 373, 8)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 92, 32)            1568      
                                                                 
 batch_normalization_1 (Batc  (None, 92, 32)           3

In [6]:
model.fit(x=X_train,
          y=y_train,
          epochs=20,
          validation_data=(X_test,y_test))

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 0.4229 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.5464 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/20
1/1 [==============================] - 0s 44ms/step - loss: 0.0345 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4624 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/20
1/1 [==============================] - 0s 45ms/step - loss: 0.0048 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.4031 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/20
1/1 [==============================] - 0s 44ms/step - loss: 0.0020 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.3591 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 45ms/step - loss: 8.3447e-04 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.3267 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 45ms/step - loss: 2.5392e-04 - precision: 1.0000 -